In [ ]:
import os
from PIL import Image
from pathlib import Path

import torch
import torchvision.models as models
import torchvision.transforms as transforms


# Загрузка предварительно обученной модели
resnet18 = models.resnet18(pretrained=True)

# Удаление последнего слоя классификации
resnet18_without_fc = torch.nn.Sequential(*list(resnet18.children())[:-1])

# Пример входного тензора
example_input = torch.rand(4, 3, 224, 224)

# Инференс
output = resnet18_without_fc(example_input)

# Вывод размерности выхода
print(output.shape)

model = resnet18_without_fc

In [ ]:
# Преобразование изображений к размеру и формату, необходимым для модели
preprocess = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# Функция получения эмбеддингов изображений
def get_images_embeddings(images_path, model):
    images = []
    for image_path in images_path:
        images.append(preprocess(Image.open(image_path)))
    image_tensor = torch.stack(images)

    with torch.no_grad():
        embeddings = model(image_tensor)

    return embeddings.squeeze()

In [ ]:
# Пути к изображениям
image_paths = sorted(Path("images").iterdir())

# Получение эмбеддингов для каждого изображения
embeddings = get_images_embeddings(image_paths, model)
print(embeddings.shape)

In [ ]:
from scipy.spatial.distance import euclidean

ind_0, ind_1 = 8, 4  # 4 / 8, 7 / 8

image_names = [image_path.name for image_path in image_paths]
# Получение евклидова расстояния между эмбеддингами
euclidean_distance = euclidean(embeddings[ind_0], embeddings[ind_1])
print(f"Евклидово между ембеддингами {image_names[ind_0]} {image_names[ind_1]}: {euclidean_distance}")

In [ ]:
normalized_embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

# Получение евклидова расстояния между нормализованными эмбеддингами
euclidean_distance = euclidean(normalized_embeddings[ind_0], normalized_embeddings[ind_1])
print(f"Евклидово между нормализованными ембеддингами {image_names[ind_0]} {image_names[ind_1]}: {euclidean_distance}")

In [ ]:
from scipy.spatial.distance import cosine

ind_0, ind_1 = 0, 2

image_names = [os.path.basename(image_path) for image_path in image_paths]
# Получение косинусного расстояния между эмбеддингами
cosine_distance = cosine(embeddings[ind_0], embeddings[ind_1])
print(f"Косинусное между ембеддингами {image_names[ind_0]} {image_names[ind_1]}: {cosine_distance}")

In [ ]:
# Получение косинусного расстояния между нормализованными эмбеддингами
cosine_distance = cosine(normalized_embeddings[ind_0], normalized_embeddings[ind_1])
print(f"Косинусное между нормализованными ембеддингами {image_names[ind_0]} {image_names[ind_1]}: {cosine_distance}")